#### LangChain Essentials Course

# LangSmith Starter

LangSmith is a built-in observability service and platform that integrates _very easily_ with LangChain. You don't _need_ to use LangSmith for this course, but it can be very helpful in understanding what is happening, _and_ we recommend using it beyond this course for general development with LangChain — with all of that in mind we would recommend spending a little bit of time to get familiar with LangSmith.

In [1]:
!pip install -qU \
  langchain-core \
  langchain-openai\
  langchain-community \
  langsmith

In [2]:
from IPython.display import Markdown, display


LangSmith does require an API key, but it comes with a generous free tier. You can sign up an account and get your API key [here](https://smith.langchain.com).

When using LangSmith, we need to setup our environment variables _and_ provide our API key, like so:

In [3]:
import os
from constant import langSmith_key  
from constant import gemini_key     


os.environ["LANGCHAIN_API_KEY"] = langSmith_key


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# customize your project name
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-langsmith-gemini"



In most cases, this is all we need to start seeing logs and traces in the [LangSmith UI](https://smith.langchain.com). By default, LangChain will trace LLM calls, chains, etc. We'll take a look at a quick example of this below.

## Default Tracing

As mentioned, LangSmith traces a lot of data without us needing to do anything. Let's see how that looks. We'll start by initializing our LLM.

In [4]:
os.environ["GOOGLE_API_KEY"] = gemini_key

from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = "gemini-1.5-flash"  

llm = ChatGoogleGenerativeAI(model=gemini_model, temperature=0.0)      
creative_llm = ChatGoogleGenerativeAI(model=gemini_model, temperature=0.9)  

print("LangSmith + Gemini setup complete!")


✅ LangSmith + Gemini setup complete!


In [6]:
import textwrap

response = llm.invoke("hello")
wrapped = textwrap.fill(response.content, width=80)
print(wrapped)

Hello there! How can I help you today?


After this we should see that a new project (`aurelioai-langchain-course-langsmith-openai`) has been created in the LangSmith UI. Inside that project, we should see the trace from our LLM call:


![LangSmith LLM Trace](../assets/langsmith_dashboard.png)

By default, LangSmith will capture plenty — however, it won't capture functions from outside of LangChain. Let's see how we can trace those.

## Tracing Non-LangChain Code


LangSmith can trace functions that are not part of LangChain, we just need to add the `@traceable` decorator. Let's try this for a few simple functions.

In [8]:
from langsmith import traceable
import random
import time


@traceable
def generate_random_number():
    return random.randint(0, 100)

@traceable
def generate_string_delay(input_str: str):
    number = random.randint(1, 5)
    time.sleep(number)
    return f"{input_str} ({number})"

@traceable
def random_error():
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "No error"

Let's run these a few times and see what happens.

In [9]:
from tqdm.auto import tqdm

for _ in tqdm(range(10)):
    generate_random_number()
    generate_string_delay("Hello boss")
    try:
        random_error()
    except ValueError:
        pass

f:\github\langChain-v0.3\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 10/10 [00:27<00:00,  2.70s/it]


Those traces should now be visible in the LangSmith UI, again under the same project:



![LangSmith UI filters](../assets/value_error.png)

There are various other things we can do within the UI, but we'll leave that for you to explore.

Finally, we can also modify our traceable names if we'd like to make them more readable inside the UI. For example:

In [10]:
from langsmith import traceable

@traceable(name="Chitchat Maker")
def error_generation_function(question: str):
    delay = random.randint(0, 3)
    time.sleep(delay)
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "I'm great how are you?"

Let's run this a few times and see what we get in LangSmith.


In [11]:
for _ in tqdm(range(10)):
    try:
        error_generation_function("How are you today?")
    except ValueError:
        pass

100%|██████████| 10/10 [00:17<00:00,  1.70s/it]


Let's filter for the `Chitchat Maker` traceable and see our results.

![LangSmith Chitchat Maker runs](../assets/chaitchat_maker.png)

We can see our runs and their related metadata! That's it for this intro to LangSmith. As we work through the course we will (optionally) refer to LangSmith for digging into our runs.